# Packing circles, maximize sum of radii



Given a positive integer $n$, the problem is to pack $n$ disjoint circles inside a unit square so as to maximize the sum of their radii. The state of the art can be found on [Erich Friedman's homepage](https://erich-friedman.github.io/packing/cirRsqu/).

* For $n=26$, the SOTA was $2.634$, and AlphaEvolve improved it to $2.635$ (Construction 1).
* For $n=32$, the SOTA was $2.936$, and AlphaEvolve improved it to $2.937$ (Construction 2).

The constructions found by AlphaEvolve are shown below.

In [ ]:
#@title Verification function

def _circles_overlap(centers, radii):
  """Protected function to compute max radii."""
  n = centers.shape[0]

  for i in range(n):
    for j in range(i + 1, n):
      dist = np.sqrt(np.sum((centers[i] - centers[j]) ** 2))
      if radii[i] + radii[j] > dist:
        return True

  return False

def check_construction(centers, radii, n) -> dict[str, float]:
  """Evaluates circle packing for maximizing sum of radii in unit square."""

  # General checks for the whole array
  if centers.shape != (n, 2) or not np.isfinite(centers).all():
    print(
        "Error: The 'centers' array has an invalid shape or non-finite values."
    )
    return {'sum_of_radii': -np.inf}

  # --- Start of the modified geometric check ---

  # 1. Check each circle individually to see if it's contained
  is_contained = (
      (radii[:, None] <= centers) & (centers <= 1 - radii[:, None])
  ).all(axis=1)

  # 2. If not all of them are contained...
  if not is_contained.all():
    return {'sum_of_radii': -np.inf}

  if (
      radii.shape != (n,)
      or not np.isfinite(radii).all()
      or not (0 <= radii).all()
  ):
    print('radii bad shape')
    return {'sum_of_radii': -np.inf}

  if _circles_overlap(centers, radii):
    print('circles overlap')
    return {'sum_of_radii': -np.inf}

  print("The circles are disjoint and lie inside the unit square.")
  return {'sum_of_radii': float(np.sum(radii))}

In [ ]:
#@title Construction 1: Data

import numpy as np

centers_26 = np.array([[0.7026096036990247 , 0.3816658444526457 ], [0.31311580997040145, 0.9076084484290411 ], [0.726905714311596 , 0.5960427019081348 ], [0.894817439731251 , 0.27395283962395217], [0.9038486659542352 , 0.6820800429325902 ], [0.10306052014158237, 0.48460080265191496], [0.2716298514860014 , 0.5976347963876618 ], [0.23971052792753603, 0.7636735693833854 ], [0.759352401560932 , 0.7629588636539982 ], [0.915073737545101 , 0.084926262454899 ], [0.4955317606743583 , 0.2753426167714399 ], [0.896532766642048 , 0.48259558221054216], [0.49728444620411005, 0.07886037291596384], [0.11077901279071568, 0.8892209872092842 ], [0.8888438205895552 , 0.8888438205895551 ], [0.49942836913903915, 0.9060726627225563 ], [0.4986680755034023 , 0.5299634197531913 ], [0.08463950069577313, 0.08463950069577317], [0.2946094887818273 , 0.1302211010652256 ], [0.4033587836026808 , 0.742417049501635 ], [0.6859430219867827 , 0.9074079050485644 ], [0.7023095250891603 , 0.13325857277081155], [0.5952197329397327 , 0.742049443460592 ], [0.09573232930702141, 0.6832585349730799 ], [0.29474605904894957, 0.38692355340959994], [0.10679014462858022, 0.2747832833508202 ]])
radii_26 = np.array([0.11514888016002278, 0.09239155157095891, 0.1006003678187114 , 0.10518256026874886, 0.09615133404576481, 0.10306052014158237, 0.09989835059275479, 0.06918067635723471, 0.06944019371125845, 0.08492626245489895, 0.11762968804654109, 0.10346723335795205, 0.07886037291596379, 0.11077901279071568, 0.11115617941044476, 0.0939273372774436 , 0.13701043012374758, 0.08463950069577311, 0.13022110106522491, 0.09584232574550489, 0.09259209495143558, 0.1332585727708113 , 0.09601897575825388, 0.09573232930702141, 0.11207708895025692, 0.10679014462858022])


In [ ]:
#@title Construction 1: Verification
print(f"Construction 1 has {len(centers_26)} circles.")
score = check_construction(centers_26, radii_26, 26)
print(f"Construction 1 sum of radii: {score['sum_of_radii']}")

In [ ]:
#@title Construction 1: Visualization

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

def visualize_packing(
    centers: np.ndarray,
    radii: np.ndarray,
    width: float,
    height: float,
    title: str = "Circle Packing"
):
    """
    Creates and displays a plot of the circle packing within a rectangle.

    Args:
        centers: An (n, 2) array of circle centers.
        radii: An (n,) array of circle radii.
        width: The width of the rectangle.
        height: The height of the rectangle.
        title: An optional title for the plot.
    """
    fig, ax = plt.subplots(1, figsize=(8, 8))

    # 1. Set the plot to have an equal aspect ratio
    ax.set_aspect('equal', adjustable='box')

    # 2. Draw the rectangle container
    rect = patches.Rectangle(
        (0, 0),
        width,
        height,
        linewidth=2,
        edgecolor='black',
        facecolor='none',
        label='Boundary'
    )
    ax.add_patch(rect)

    # 3. Draw each circle and label it with its index
    for i in range(len(radii)):
        circle = patches.Circle(
            (centers[i, 0], centers[i, 1]),
            radii[i],
            edgecolor='royalblue',
            facecolor='skyblue',
            alpha=0.6
        )
        ax.add_patch(circle)
        # Add the index number 'i' to the center of the circle
        ax.text(centers[i, 0], centers[i, 1], str(i),
                ha='center', va='center', fontsize=8, color='navy')

    # 4. Set plot limits and appearance
    margin = max(width, height) * 0.05  # 5% margin
    ax.set_xlim(-margin, width + margin)
    ax.set_ylim(-margin, height + margin)
    ax.set_title(title, fontsize=16)
    ax.set_xlabel("Width")
    ax.set_ylabel("Height")

    # 5. Show the plot
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.show()

visualize_packing(centers_26, radii_26, 1, 1, "AlphaEvolve's 26-Circle Packing")

In [ ]:
#@title Construction 2: Data

centers_32 = np.array([[0.4103367574347774 , 0.4034775285334993 ], [0.8963458552602112 , 0.10365414473978883], [0.2466174786966004 , 0.6773614002482867 ], [0.40962670073476165, 0.21822588350952354], [0.24333518337797744, 0.1079405010782716 ], [0.07306696147175942, 0.9269330385282406 ], [0.09052877125190313, 0.5853168363356677 ], [0.5726977383625569 , 0.106228715042693 ], [0.7369697857091396 , 0.21625157750836602], [0.573911802908532 , 0.3081330707478824 ], [0.4086745937675105 , 0.06331525320633652], [0.7355143578272628 , 0.06238720755320459], [0.25728850226113253, 0.8838819371737265 ], [0.8884332917756496 , 0.8884332917756496 ], [0.739436872889558 , 0.4026491108074175 ], [0.572899637814724 , 0.7133709992493075 ], [0.24844240778977586, 0.3094037922636657 ], [0.5743914194083511 , 0.5010189546480183 ], [0.09056937488828318, 0.4042186947473096 ], [0.9061987946708536 , 0.6838352301421212 ], [0.0875712557500808 , 0.2261032951720141 ], [0.9034277921600417 , 0.30375521529448646], [0.3904454681243297 , 0.7519411666029409 ], [0.4712803348377272 , 0.9014096013683925 ], [0.7562770574133956 , 0.75857067081745 ], [0.08942465018367278, 0.7652668705202621 ], [0.06977107568297637, 0.06977107568297637], [0.7427641440618451 , 0.5914750415187353 ], [0.9051413826149427 , 0.4951783707749125 ], [0.2488889588844859 , 0.494844162673007 ], [0.6665042606822044 , 0.90335676254708 ], [0.4068368918378875 , 0.5892639181130421 ]])
radii_32 = np.array([0.0936547028434594 , 0.1036541447397888 , 0.0906779790021106 , 0.09159830297454706, 0.10794050107827158, 0.07306696147175941, 0.09052877125190312, 0.10622871504269298, 0.09148404581147725, 0.09567929075561458, 0.0633152532063365 , 0.06238720755320458, 0.11611806282627339, 0.11156670822435043, 0.0949298134885404 , 0.11515009501531538, 0.09358751542673785, 0.09720718943396116, 0.09056937488828316, 0.09380120532914636, 0.08757125575008079, 0.09657220783995835, 0.07133631694839435, 0.09859039863160741, 0.07371569947520423, 0.08942465018367277, 0.06977107568297636, 0.09392542960110564, 0.09485861738505727, 0.09185339264220439, 0.09664323745291996, 0.09216464924936743])


In [ ]:
#@title Construction 2: Verification
print(f"Construction 2 has {len(centers_32)} circles.")
score = check_construction(centers_32, radii_32, 32)
print(f"Construction 2 sum of radii: {score['sum_of_radii']}")

In [ ]:
#@title Construction 2: Visualization
visualize_packing(centers_32, radii_32, 1, 1, "AlphaEvolve's 32-Circle Packing")

In [ ]:
#@title Initial program used


def construct_packing(num_circles):
  """Searches for the best packing of circles within a unit square to maximize sum of radii."""
  variable_name_radii = f'radii_{num_circles}'
  variable_name_centers = f'centers_{num_circles}'
  if np.random.rand() < 0.5 and variable_name_radii in globals():
    radii = globals()[variable_name_radii]
    centers = globals()[variable_name_centers]
    claimed_score = np.sum(radii)
    placed_circles_params = (centers, radii)
  else:
    centers = np.random.rand(num_circles, 2)
    radii = np.random.rand(num_circles) / 10000.0
    claimed_score = np.sum(radii)
    placed_circles_params = (centers, radii)
  best_score = 0  # placeholder, have to compute this
  best_placements_params = list(placed_circles_params).copy()
  print(f'Initial score: {best_score}, placements: {placed_circles_params}')
  return (best_placements_params[0], best_placements_params[1], best_score)
  # return initial construction for now, but this should be changed
  start_time = time.time()
  eval_count = 0
  while time.time() - start_time < 1000:
    index_to_mutate = np.random.randint(0, len(placed_circles_params))

    centers[index_to_mutate] = np.random.rand(2)
    radii[index_to_mutate] = np.random.rand()
    placed_circles_params = (centers, radii)
    score = 0  # placeholder, have to compute this
    eval_count += 1
    if score > best_score:
      best_score = score
      best_placements_params = list(placed_circles_params).copy()
      print(
          f'best score: {best_score}, best_placements: {best_placements_params}'
      )
    if np.random.rand() < 0.2:
      placed_circles_params = list(best_placements_params).copy()

  print(f'Final score: {best_score}')
  print(f'Evaluations: {eval_count}')
  return (best_placements_params[0], best_placements_params[1], best_score)

In [ ]:
#@title Code evolved by AlphaEvolve

"""Packs circles to maximize sum of radii in unit square."""
import itertools
import logging
import time
from scipy import integrate
import numpy as np
from scipy import optimize
import warnings
import random
import re
from typing import Any, Callable, Mapping, List, Tuple
import scipy.linalg as la
import collections
import copy
import math
import numba

njit = numba.njit



# --- Numba-accelerated helper functions for SLSQP ---
@njit
def constraints_bh_fast(vars, N):
    centers_x = vars[0::3]
    centers_y = vars[1::3]
    radii = vars[2::3]

    num_constraints = 4 * N + N * (N - 1) // 2
    constraints = np.empty(num_constraints, dtype=np.float64)

    # 1. Boundary constraints (vectorized slices for Numba)
    constraints[0:N] = centers_x - radii
    constraints[N:2*N] = 1.0 - centers_x - radii
    constraints[2*N:3*N] = centers_y - radii
    constraints[3*N:4*N] = 1.0 - centers_y - radii

    # 2. Overlap constraints
    offset = 4 * N
    k = 0
    for i in range(N):
        for j in range(i + 1, N):
            dist_sq = (centers_x[i] - centers_x[j])**2 + (centers_y[i] - centers_y[j])**2
            r_sum = radii[i] + radii[j]
            constraints[offset + k] = dist_sq - r_sum * r_sum
            k += 1

    return constraints

@njit
def jac_constraints_bh_fast(vars, N):
    centers_x = vars[0::3]
    centers_y = vars[1::3]
    radii = vars[2::3]

    num_constraints = 4 * N + N * (N - 1) // 2
    jac = np.zeros((num_constraints, 3 * N), dtype=np.float64)

    # 1. Boundary constraints (4N) in a single loop for efficiency
    for i in range(N):
        # C1: x_i - r_i >= 0
        jac[i, 3*i] = 1.0
        jac[i, 3*i+2] = -1.0
        # C2: 1 - x_i - r_i >= 0
        jac[N+i, 3*i] = -1.0
        jac[N+i, 3*i+2] = -1.0
        # C3: y_i - r_i >= 0
        jac[2*N+i, 3*i+1] = 1.0
        jac[2*N+i, 3*i+2] = -1.0
        # C4: 1 - y_i - r_i >= 0
        jac[3*N+i, 3*i+1] = -1.0
        jac[3*N+i, 3*i+2] = -1.0

    # 2. Overlap constraints
    offset = 4 * N
    k = 0
    for i in range(N):
        for j in range(i + 1, N):
            dx = centers_x[i] - centers_x[j]
            dy = centers_y[i] - centers_y[j]
            dr_sum = radii[i] + radii[j]

            idx = offset + k

            # Derivatives w.r.t circle i
            jac[idx, 3*i] = 2.0 * dx
            jac[idx, 3*i+1] = 2.0 * dy
            jac[idx, 3*i+2] = -2.0 * dr_sum

            # Derivatives w.r.t circle j
            jac[idx, 3*j] = -2.0 * dx
            jac[idx, 3*j+1] = -2.0 * dy
            jac[idx, 3*j+2] = -2.0 * dr_sum

            k += 1

    return jac

def construct_packing(num_circles):
  """Searches for the best packing of circles within a unit square to maximize sum of radii."""
  variable_name_radii = f'radii_{num_circles}'
  variable_name_centers = f'centers_{num_circles}'
  # Always start from the previous best solution if available.
  # Basin Hopping is designed to explore from the starting point.
  if variable_name_radii in globals():
    radii = globals()[variable_name_radii]
    centers = globals()[variable_name_centers]
    claimed_score = np.sum(radii)
  else:
    # Start with a structured packing for a better initial guess if no prior solution exists.
    if num_circles == 1:
        centers = np.array([[0.5, 0.5]])
        radii = np.array([0.5])
    else:
        # Arrange circles in a grid-like pattern
        # Determine grid dimensions for approximately square arrangement
        rows = int(np.ceil(np.sqrt(num_circles)))
        cols = int(np.ceil(num_circles / rows))

        # Calculate a reasonable initial radius based on grid size
        # Assuming circles fill the grid cells
        initial_r_estimate = 0.5 / max(rows, cols)

        centers = np.zeros((num_circles, 2))
        radii = np.full(num_circles, initial_r_estimate)

        k = 0
        for i in range(rows):
            for j in range(cols):
                if k < num_circles:
                    # Place centers at the middle of grid cells
                    x_center = (j + 0.5) / cols
                    y_center = (i + 0.5) / rows
                    centers[k, 0] = x_center
                    centers[k, 1] = y_center
                    k += 1
                else:
                    break

        # Ensure radii are non-negative and provide a small buffer
        # The initial_r_estimate might lead to overlaps if grid is too tight,
        # so slightly shrink or ensure a minimum.
        radii = np.maximum(radii * 0.95, 1e-5) # Shrink slightly, ensure min radius.
    claimed_score = np.sum(radii)

  best_score = claimed_score
  # Ensure we have copies so we don't modify the global variables if they were loaded
  best_centers = centers.copy()
  best_radii = radii.copy()
  n = num_circles

  # Helper function to check validity of a configuration
  @njit
  def is_valid(centers, radii):
    N = len(radii)
    # Check boundaries: [0, 1] x [0, 1]
    for i in range(N):
      r = radii[i]
      if r < 0: return False
      x, y = centers[i]
      if x < r or x > 1.0 - r or y < r or y > 1.0 - r:
        return False

    # Check overlaps
    for i in range(N):
      for j in range(i + 1, N):
        dist_sq = (centers[i, 0] - centers[j, 0])**2 + (centers[i, 1] - centers[j, 1])**2
        # If (r_i + r_j)^2 > dist_sq, there is an overlap.
        if dist_sq < (radii[i] + radii[j])**2:
          return False
    return True

  if not is_valid(best_centers, best_radii):
      print("Warning: Initial configuration (potentially from globals) is invalid. Attempting to fix.")
      # If it's invalid, try to fix it by slight shrinkage before completely resetting.
      # This preserves some of the prior optimization effort.
      # Align tolerance with the expected precision from SLSQP.
      # For initial fix, a slightly more aggressive tolerance is fine.
      TOLERANCE = 1e-12 # More aggressive than the ftol, but not too small to lose valid radii.
      fixed_radii = best_radii * (1.0 - TOLERANCE)
      if is_valid(best_centers, fixed_radii):
          best_radii = fixed_radii
          best_score = np.sum(best_radii)
          print(f"Fixed initial invalid configuration. New score: {best_score:.6f}")
      else:
          # If a simple shrinkage doesn't fix it, attempt a quick full optimization
          # from the slightly shrunk state before resorting to trivial solution.
          print("Warning: Initial shrinkage failed to fix. Attempting quick initial optimization.")
          initial_vars = np.zeros(3 * n)
          initial_vars[0::3] = best_centers[:, 0]
          initial_vars[1::3] = best_centers[:, 1]
          initial_vars[2::3] = fixed_radii # Use the shrunk radii

          # Use a relaxed ftol and limited iterations for this initial "fix" run.
          initial_fix_res = optimize.minimize(objective_bh, initial_vars,
                                          method='SLSQP',
                                          jac=jac_objective_bh,
                                          bounds=bounds,
                                          constraints=minimizer_kwargs['constraints'],
                                          options={'maxiter': 1000, 'ftol': 1e-10, 'disp': False})

          fixed_centers_opt = np.zeros((n, 2))
          fixed_centers_opt[:, 0] = initial_fix_res.x[0::3]
          fixed_centers_opt[:, 1] = initial_fix_res.x[1::3]
          fixed_radii_opt = initial_fix_res.x[2::3]
          fixed_score_opt = -initial_fix_res.fun

          # After this quick opt, check validity and score.
          if is_valid(fixed_centers_opt, fixed_radii_opt) and fixed_score_opt > np.sum(np.full(n, 1e-5)): # Must be better than trivial
              best_centers = fixed_centers_opt
              best_radii = fixed_radii_opt
              best_score = fixed_score_opt
              print(f"Quick initial optimization fixed configuration. New score: {best_score:.6f}")
          else:
              print("Warning: Could not fix initial invalid configuration with quick opt. Resetting to trivial solution.")
              best_centers = np.random.rand(n, 2) * 0.5 + 0.25 # Start somewhat centered
              best_radii = np.full(n, 1e-5)
              best_score = np.sum(best_radii)

  print(f'Initial score: {best_score:.6f}')

  start_time = time.time()
  time_limit = 995.0 # Leave some buffer time
  eval_count = 0

  # Optimization strategy: Basin Hopping with SLSQP for local optimization.

  # --- Optimization Setup ---

  # Objective function: minimize negative sum of radii
  def objective_bh(vars):
    # vars = [x1, y1, r1, x2, y2, r2, ...]
    radii = vars[2::3]
    return -np.sum(radii)

  # Gradient of the objective function
  def jac_objective_bh(vars):
    # Objective: -sum(r_i)
    N_vars = len(vars)
    grad = np.zeros(N_vars)
    # d/dr_i = -1
    grad[2::3] = -1.0
    return grad

  # Constraints function for SLSQP (C(x) >= 0)
  # Using squared distances for better numerical stability and gradients.
  def constraints_bh(vars):
    N = len(vars) // 3
    return constraints_bh_fast(vars, N)

  # Jacobian of the constraints
  def jac_constraints_bh(vars):
    N = len(vars) // 3
    return jac_constraints_bh_fast(vars, N)

  # Bounds for variables (x, y, r) -> [0, 1], [0, 1], [0, 0.5]
  bounds = [(0.0, 1.0), (0.0, 1.0), (0.0, 0.5)] * n

  # Minimizer arguments for SLSQP
  # Tuning ftol and maxiter balances speed and accuracy of local search.
  # Providing analytical Jacobian (jac) significantly speeds up SLSQP.
  minimizer_kwargs = {
      "method": "SLSQP",
      "bounds": bounds,
      "jac": jac_objective_bh,
      "constraints": {'type': 'ineq', 'fun': constraints_bh, 'jac': jac_constraints_bh},
      # With analytical gradients, we can afford very high precision.
      # Increasing precision (decreasing ftol) and maxiter is crucial for refining the solution.
      # We push for extreme precision to get improvements in the last digits.
      "options": {'maxiter': 15000, 'ftol': 1e-16, 'disp': False}
  }

  # Vector of all variables [x1, y1, r1, x2, y2, r2, ...].
  best_vars = np.zeros(3 * n)
  best_vars[0::3] = np.clip(best_centers[:, 0], 0.0, 1.0)
  best_vars[1::3] = np.clip(best_centers[:, 1], 0.0, 1.0)
  best_vars[2::3] = np.clip(best_radii, 0.0, 0.5)

  # --- Perturbation setup ---
  avg_r_estimate = 1.0 / (2.0 * np.sqrt(n)) if n > 0 else 0.5
  # Slightly smaller base steps, as we expect to be near optimum.
  stepsize_pos = avg_r_estimate * 0.05
  stepsize_rad = avg_r_estimate * 0.025
  bounds_arr = np.array(bounds)

  # --- Run Optimization ---
  print("Starting custom iterative optimization...")
  best_vars = np.zeros(3 * n)
  best_vars[0::3] = best_centers[:, 0]
  best_vars[1::3] = best_centers[:, 1]
  best_vars[2::3] = best_radii

  # Main optimization loop
  while time.time() - start_time < time_limit:
    eval_count += 1
    time_elapsed = time.time() - start_time
    time_fraction = time_elapsed / time_limit
    current_vars = best_vars.copy()

    # --- Adaptive optimization parameters ---
    # ftol: starts looser, gets tighter. Minimum 1e-16 (for main loop).
    # This uses a more aggressive exponential decay, but caps at a minimum.
    current_ftol = max(1e-16, 1e-8 * np.exp(-10.0 * time_fraction))

    # Perturbation step scale: decays exponentially.
    step_scale = np.exp(-5.0 * time_fraction) # Decays from 1 (start) to ~0.0067 (end)
    current_stepsize_pos = stepsize_pos * step_scale
    current_stepsize_rad = stepsize_rad * step_scale

    # Maxiter for SLSQP: increases over time for more thorough local search.
    # The base maxiter is increased slightly.
    base_maxiter = 5000
    maxiter_ramp = 10000
    current_maxiter_full = int(base_maxiter + maxiter_ramp * time_fraction)
    current_maxiter_subset = int(base_maxiter * 0.6 + maxiter_ramp * 0.6 * time_fraction) # Smaller for subsets

    # --- Dynamic Strategy Selection ---
    rand_val = np.random.rand()

    # Probability for full optimization. Higher for small N, decreases over time.
    # Small N (<= 5) gets a higher chance of full optimization.
    prob_full_base = 0.05
    prob_full_small_n_bonus = 0.15 if n <= 5 else 0.0
    prob_full = (prob_full_base + prob_full_small_n_bonus) * (1.0 - time_fraction * 0.5) # Decay slightly over time
    prob_full = min(1.0, max(0.05, prob_full)) # Ensure it's within reasonable bounds

    # Probability for global perturbation (shaking all circles) followed by full refinement.
    # More prominent early on for exploration.
    prob_global_perturb = 0.05 * (1.0 - time_fraction)

    # Probability for subset perturbation.
    prob_subset_perturb = 0.3 + 0.2 * (1.0 - time_fraction) # Starts higher, decays.

    # Remaining probability goes to Subset Refinement without initial perturbation.

    # Strategy 1: Full Refinement
    if rand_val < prob_full:
      res = optimize.minimize(objective_bh, current_vars,
                              method='SLSQP',
                              jac=jac_objective_bh,
                              bounds=bounds,
                              constraints=minimizer_kwargs['constraints'],
                              options={'maxiter': current_maxiter_full, 'ftol': current_ftol, 'disp': False})
      x_new = res.x

    # Strategy 2: Global Perturbation then Full Refinement
    elif rand_val < prob_full + prob_global_perturb:
        base_vars_perturb = current_vars.copy()
        for i in range(n):
            base_vars_perturb[3*i:3*i+2] += np.random.normal(0, current_stepsize_pos / 2.0, size=2) # Larger perturbation
            base_vars_perturb[3*i+2] += np.random.normal(0, current_stepsize_rad / 2.0)
        # Clip to stay within bounds
        np.clip(base_vars_perturb, bounds_arr[:, 0], bounds_arr[:, 1], out=base_vars_perturb)

        res = optimize.minimize(objective_bh, base_vars_perturb,
                                method='SLSQP',
                                jac=jac_objective_bh,
                                bounds=bounds,
                                constraints=minimizer_kwargs['constraints'],
                                options={'maxiter': int(current_maxiter_full * 0.75), 'ftol': current_ftol, 'disp': False}) # Slightly fewer iterations after global perturb
        x_new = res.x

    else: # Strategies 3 and 4 operate on subsets.
      base_vars = current_vars.copy() # Make a copy for subset operations

      # Determine if this is a perturbation step (Strategy 3) or pure refinement (Strategy 4)
      is_perturbation_step = rand_val < prob_full + prob_global_perturb + prob_subset_perturb

      # Adaptive subset size: larger early, smaller late
      # Ensure min 2 circles (if n>1) and not the full set
      subset_size_early = max(2, int(n * 0.3))
      subset_size_late = max(2, int(n * 0.1))
      subset_size_calculated = int(subset_size_early * (1.0 - time_fraction) + subset_size_late * time_fraction)
      subset_size = min(n - 1, max(2, subset_size_calculated))
      if n == 1: subset_size = 1 # Handle n=1 edge case for subset size

      # --- Targeted Subset Selection (Neighborhood-based and radius-based) ---
      # Focus on optimizing clusters of circles that are tightly constrained or have large radii.
      constraint_values = constraints_bh_fast(base_vars, n)
      TIGHT_TOL = 1e-8
      active_circles = set()
      adj = collections.defaultdict(list)

      # Find active circles from boundary constraints
      for i in range(n):
        if any(constraint_values[k*n + i] < TIGHT_TOL for k in range(4)):
          active_circles.add(i)

      # Find active circles from overlap constraints and build adjacency list
      offset = 4 * n
      k = 0
      for i in range(n):
        for j in range(i + 1, n):
          if constraint_values[offset + k] < TIGHT_TOL:
            active_circles.add(i)
            active_circles.add(j)
            adj[i].append(j)
            adj[j].append(i)
          k += 1

      subset_indices = []
      if not active_circles or n==1: # If no active circles or only one, just pick random.
          subset_indices = sorted(np.random.choice(n, size=subset_size, replace=False).tolist())
      else:
          # Build subset from connected components of active circles
          active_circles_list = list(active_circles)
          random.shuffle(active_circles_list)
          visited = set()

          # Prioritize components by size or by average radius of circles within.
          components = []
          for seed in active_circles_list:
              if seed in visited:
                  continue

              component = []
              q = collections.deque([seed])
              visited.add(seed)
              while q:
                  current_node = q.popleft()
                  component.append(current_node)
                  for neighbor in adj[current_node]:
                      if neighbor not in visited:
                          visited.add(neighbor)
                          q.append(neighbor)
              components.append(component)

          # Sort components by sum of radii in descending order to prioritize larger/more important ones
          components.sort(key=lambda comp: sum(base_vars[3*c+2] for c in comp), reverse=True)

          for component in components:
              if len(subset_indices) >= subset_size:
                  break

              needed = subset_size - len(subset_indices)
              if len(component) <= needed:
                  subset_indices.extend(component)
              else:
                  # Take the largest circles from the component if it's too big
                  component_radii = [(base_vars[3*c+2], c) for c in component]
                  component_radii.sort(key=lambda x: x[0], reverse=True) # Sort by radius, descending
                  subset_indices.extend([c for r, c in component_radii[:needed]])

          # If we still don't have enough circles, fill the rest randomly,
          # prioritizing larger circles among the non-selected ones.
          if len(subset_indices) < subset_size:
              remaining_needed = subset_size - len(subset_indices)
              other_circles_with_radii = [(base_vars[3*i+2], i) for i in range(n) if i not in subset_indices]
              other_circles_with_radii.sort(key=lambda x: x[0], reverse=True) # Sort by radius, descending

              num_to_add = min(remaining_needed, len(other_circles_with_radii))
              subset_indices.extend([circle_idx for radius, circle_idx in other_circles_with_radii[:num_to_add]])

          subset_indices = sorted(list(set(subset_indices)))

      # Strategy 3: Perturb Subset before Refinement
      if is_perturbation_step:
          for i in subset_indices:
              base_vars[3*i:3*i+2] += np.random.normal(0, current_stepsize_pos / 3.0, size=2)
              base_vars[3*i+2] += np.random.normal(0, current_stepsize_rad / 3.0)

          # Clip perturbed variables to stay within bounds
          col_indices_to_clip = np.concatenate([np.arange(3*i, 3*i+3) for i in subset_indices])
          np.clip(base_vars[col_indices_to_clip], bounds_arr[col_indices_to_clip, 0], bounds_arr[col_indices_to_clip, 1], out=base_vars[col_indices_to_clip])

      # --- Common Subset Optimization Logic ---
      subset_x0 = np.concatenate([base_vars[3*i : 3*i+3] for i in subset_indices])

      # Wrapper functions capture `base_vars` and `subset_indices` from the local scope
      # These wrappers are necessary because SLSQP's `fun`, `jac`, `bounds`, `constraints`
      # need to be defined relative to the subset_x (the variables being optimized in *this* call)
      # while still being able to access the full system state (`base_vars` and `n`).
      def subset_objective(subset_x):
          temp_vars = base_vars.copy()
          for k, i in enumerate(subset_indices):
              temp_vars[3*i : 3*i+3] = subset_x[3*k : 3*k+3]
          return objective_bh(temp_vars)

      def subset_jac_objective(subset_x):
          grad = np.zeros_like(subset_x)
          # Only radii components contribute to the objective gradient (-1.0)
          grad[2::3] = -1.0
          return grad

      def subset_constraints(subset_x):
          temp_vars = base_vars.copy()
          for k, i in enumerate(subset_indices):
              temp_vars[3*i : 3*i+3] = subset_x[3*k : 3*k+3]
          return constraints_bh_fast(temp_vars, n)

      def subset_jac_constraints(subset_x):
          temp_vars = base_vars.copy()
          for k, i in enumerate(subset_indices):
              temp_vars[3*i : 3*i+3] = subset_x[3*k : 3*k+3]
          full_jac = jac_constraints_bh_fast(temp_vars, n)

          # Filter the full Jacobian to only include columns corresponding to the subset variables
          col_indices = np.concatenate([np.arange(3*i, 3*i+3) for i in subset_indices])
          return full_jac[:, col_indices]

      col_indices_bounds = np.concatenate([np.arange(3*i, 3*i+3) for i in subset_indices])
      subset_bounds = [bounds[i] for i in col_indices_bounds]

      res = optimize.minimize(subset_objective, subset_x0,
                              method='SLSQP',
                              jac=subset_jac_objective,
                              bounds=subset_bounds,
                              constraints={'type': 'ineq', 'fun': subset_constraints, 'jac': subset_jac_constraints},
                              options={'maxiter': current_maxiter_subset, 'ftol': current_ftol, 'disp': False})

      # Reconstruct the full variable vector from the base vector
      x_new = base_vars.copy()
      for k, i in enumerate(subset_indices):
          x_new[3*i : 3*i+3] = res.x[3*k : 3*k+3]

    # --- Process Result ---
    # It's important to re-check the score and validity after *any* optimization,
    # as SLSQP might return a slightly invalid solution or not improve the score.
    score = -res.fun # Objective function minimizes negative sum of radii, so -res.fun is sum of radii.

    # Extract centers and radii from the optimized variables (x_new)
    temp_centers = np.zeros((n, 2))
    temp_centers[:, 0] = x_new[0::3]
    temp_centers[:, 1] = x_new[1::3]
    temp_radii = x_new[2::3]

    # Check validity rigorously. SLSQP can slightly violate constraints.
    # We attempt to fix by slightly shrinking radii if invalid, matching ftol precision.
    if not is_valid(temp_centers, temp_radii):
        # If invalid, try to shrink radii slightly to make it valid.
        # The tolerance for shrinkage should align with the optimization's ftol,
        # or be slightly more aggressive if ftol is already very small.
        # Using current_ftol as a basis for TOLERANCE here.
        TOLERANCE = current_ftol * 1e-2 # A bit tighter than ftol itself
        if TOLERANCE < 1e-18: TOLERANCE = 1e-18 # Ensure a minimum shrinkage

        fixed_radii = temp_radii * (1.0 - TOLERANCE)
        fixed_score = np.sum(fixed_radii)

        if is_valid(temp_centers, fixed_radii) and fixed_score > best_score:
            best_score = fixed_score
            best_centers = temp_centers.copy()
            best_radii = fixed_radii.copy()
            best_vars = x_new.copy()
            best_vars[2::3] = fixed_radii # Update radii in best_vars
            elapsed = time.time() - start_time
            print(f"New best score (fixed invalid): {best_score:.7f} at {elapsed:.2f}s (iter {eval_count})")
        # else:
        #     # If shrinking doesn't help or leads to a worse score,
        #     # we discard this result and continue with best_vars.
        #     # This means x_new is not stored in best_vars, current_vars remains the reference.
        #     pass
    elif score > best_score:
        # Solution is valid and better than current best.
        best_score = score
        best_centers = temp_centers.copy()
        best_radii = temp_radii.copy()
        best_vars = x_new.copy() # Update best_vars
        elapsed = time.time() - start_time
        print(f"New best score: {best_score:.7f} at {elapsed:.2f}s (iter {eval_count})")
    # else:
    #     # Solution is valid but not better, or invalid and not fixable to be better.
    #     # Continue with existing best_vars.
    #     pass

  # --- Final Polish ---
  # If time permits, run one last full, high-precision optimization.
  time_left = time_limit - (time.time() - start_time)
  if time_left > 30.0: # Use a minimum time budget for the final polish
      print(f"Performing final high-precision polish ({time_left:.1f}s remaining)...")
      # Scale iterations with problem size, but cap it to avoid timeouts.
      # Scale iterations with problem size, but cap it to avoid timeouts.
      # Increased base iterations and cap for the final polish.
      polish_max_iter = min(30000, 15000 + int(n * 150))
      res = optimize.minimize(objective_bh, best_vars,
                              method='SLSQP',
                              jac=jac_objective_bh,
                              bounds=bounds,
                              constraints=minimizer_kwargs['constraints'],
                              options={'maxiter': polish_max_iter, 'ftol': 1e-18, 'disp': False})

      score = -res.fun
      if score > best_score:
          centers_final = np.zeros((n, 2))
          centers_final[:, 0] = res.x[0::3]
          centers_final[:, 1] = res.x[1::3]
          radii_final = res.x[2::3]

          if is_valid(centers_final, radii_final):
              best_score = score
              best_centers = centers_final.copy()
              best_radii = radii_final.copy()
              best_vars = res.x.copy() # Update best_vars as well
              print(f"Polishing improved score to: {best_score:.7f}")

  # --- Finalization ---

  # One final check on the validity of the best solution found.
  if not is_valid(best_centers, best_radii):
      print("Warning: Final best solution is invalid. This should ideally not happen. Attempting final aggressive fix.")
      # Use a tolerance consistent with the highest SLSQP precision (ftol=1e-18).
      TOLERANCE = 1e-18
      fixed_radii = best_radii * (1.0 - TOLERANCE)
      if is_valid(best_centers, fixed_radii):
          best_radii = fixed_radii
          best_score = np.sum(best_radii)
      else:
          print("Error: Could not fix the invalid solution. Returning a trivial valid solution.")
          # Return a safe trivial solution if everything failed.
          centers = np.random.rand(n, 2) * 0.5 + 0.25
          radii = np.full(n, 1e-5)
          return centers, radii, np.sum(radii)

  print(f'Final score: {best_score:.6f}')
  print(f'Total evaluations (Basin Hopping iterations): {eval_count}')
  return (best_centers, best_radii, best_score)


**Prompt used**

Act as an expert software developer. Your task is to iteratively improve the provided codebase. Your task is to write a search function that has 1000 seconds to find a way to place num_circles disjoint disks into the unit square [0,1] x [0,1] in such a way that the sum of their radii is as big as possible.

Your program will be evaluated with the command:
centers, radii, claimed_score = construct_packing(num_circles)

So you have to write a construct_packing(num_circles) function that returns three things:
- radii is an array of num_circles non-negative, finite numbers,
- centers must be a 2D NumPy array of shape (n,2), where each of the n rows contains an (x, y) coordinate pair for the center of a circle,
- claimed_score is the sum of the radii of the circles.

Remember, the circles must be disjoint. This will be checked automatically, with a snippet as follows:
for i in range(n):
  for j in range(i + 1, n):
    dist = np.sqrt(np.sum((centers[i] - centers[j]) ** 2))
    if radii[i] + radii[j] > dist:
      return False

Your code has to complete its search in a maximum of 1000 seconds. You may start your search from the previously found best construction, which will be available to you in the centers_26 and radii_26 global variables, where of course 26 should be replaced by num_circles.

You got this! I believe in you!!!

## Packing circles inside a rectangle of perimeter 4 to maximize sum of radii

Given a positive integer $n$, the problem is to pack $n$ disjoint circles inside a rectangle of perimeter 4 so as to maximize the sum of their radii. The state of the art can be found on [Erich Friedman's homepage](https://erich-friedman.github.io/packing/cirRrec/).

For $n=21$, the SOTA was $2.364$, and AlphaEvolve improved it to $2.365$. The construction found by AlphaEvolve is shown below.

In [ ]:
#@title Data
import numpy as np

centers_21 = np.array([[0.5116344482349532 , 0.6042884846390946 ], [0.7087426256542229 , 0.7175624677635186 ], [0.3145262708156835 , 0.7175624677635184 ], [0.31647214265648654, 0.9019896034582606 ], [0.899307288731929 , 0.8527694957921154 ], [0.7141245569697637 , 0.4958995816816336 ], [0.29193030461281383, 0.29820323112068375], [0.11292169820594489, 0.3858951194945234 ], [0.30914433950014253, 0.49589958168163345], [0.6306973045788548 , 0.11906285634390201], [0.7313385918570933 , 0.29820323112068287], [0.9081857702350873 , 0.6138896988097773 ], [0.7067967538134197 , 0.9019896034582613 ], [0.1370712971165672 , 0.13707129711656757], [0.9103471982639619 , 0.3858951194945241 ], [0.1239616077379781 , 0.8527694957921159 ], [0.5116344482349533 , 0.8493309125983606 ], [0.5116344482349529 , 0.34782102271875326], [0.39257159189105156, 0.11906285634390204], [0.8861975993533394 , 0.13707129711656638], [0.11508312623481891, 0.6138896988097775 ]])
radii_21 = np.array([0.11764223702753107, 0.10969590068459595, 0.10969590068459542, 0.07474150007183197, 0.12396160773797633, 0.11203231188844417, 0.08641206324529244, 0.112921698205944 , 0.11203231188844416, 0.11906285634390094, 0.08641206324529362, 0.11508312623481771, 0.0747415000718321 , 0.13707129711656585, 0.11292169820594324, 0.12396160773797653, 0.12740019093173208, 0.13882522489280846, 0.11906285634390067, 0.13707129711656568, 0.11508312623481826])
width_21 = np.float64(1.0232688964699064)
height_21 = np.float64(0.9767311035300936)



In [ ]:
#@title Verification
import itertools

def _circles_overlap(centers, radii):
  """Protected function to compute max radii."""
  n = centers.shape[0]

  for i in range(n):
    for j in range(i + 1, n):
      dist = np.sqrt(np.sum((centers[i] - centers[j]) ** 2))
      if radii[i] + radii[j] > dist:
        return True

  return False

import numpy as np
import itertools


import numpy as np
import itertools


def check_construction_rectangle(centers: np.ndarray, radii: np.ndarray, n: int, width: float, height: float) -> dict[str, float]:
  """
  Evaluates a circle packing in a rectangle.

  Checks if all circles are contained within the rectangle and do not overlap.
  Provides detailed diagnostics for any violations, distinguishing between
  genuine errors and potential floating-point precision issues.

  Args:
    centers: A numpy array of shape (n, 2) with the (x, y) coordinates of the circle centers.
    radii: A numpy array of shape (n,) with the radii of the circles.
    n: The number of circles.
    width: The width of the rectangle.
    height: The height of the rectangle.

  Returns:
    A dictionary containing the sum of radii if the packing is valid,
    or -np.inf if it is invalid.
  """

  TOLERANCE = 1e-9 # Tolerance for floating-point comparisons

  # --- Start of checks for rectangle geometry ---
  # 1. Check if width and height are finite, real numbers.
  if not np.all(np.isfinite([width, height])) or not np.isrealobj(
      np.array([width, height])
  ):
    print(f'Error: Invalid width or height. Must be finite real numbers.')
    return {'sum_of_radii': -np.inf}

  # 2. Check if the rectangle's perimeter is 4.
  if not np.isclose(2 * (width + height), 4.0):
    print(f'Error: Perimeter is not 4. Got {2 * (width + height)}')
    return {'sum_of_radii': -np.inf}

  # 3. Check for valid, non-degenerate rectangle dimensions.
  if width <= 0 or height <= 0:
    print(
        f'Error: Invalid rectangle dimensions. width={width}, height={height}'
    )
    return {'sum_of_radii': -np.inf}
  # --- End of rectangle checks ---

  # General checks for the input arrays
  if (
      centers.shape != (n, 2)
      or not np.isfinite(centers).all()
      or not np.isrealobj(centers)
  ):
    print(
        "Error: The 'centers' array has an invalid shape, non-finite, or"
        ' complex values.'
    )
    return {'sum_of_radii': -np.inf}

  # --- Geometric check for circle containment ---
  # 1. Check each circle individually to see if it's contained
  is_contained = (
      (radii[:, None] <= centers)
      & (centers <= np.array([width, height]) - radii[:, None])
  ).all(axis=1)

  # 2. If not all of them are contained, print diagnostics
  if not is_contained.all():
    print('Error: Circles are not contained within the rectangle.')
    for i, contained in enumerate(is_contained):
      if not contained:
        print(f'-> Diagnostics for Circle {i}:')
        c_i = centers[i]
        r_i = radii[i]
        # Check violation for each of the four boundaries
        violations = {
            'left': r_i - c_i[0],
            'right': c_i[0] - (width - r_i),
            'bottom': r_i - c_i[1],
            'top': c_i[1] - (height - r_i),
        }
        for boundary, violation_amount in violations.items():
          if violation_amount > TOLERANCE:
            print(f'  - Genuinely violates {boundary} boundary by {violation_amount:.4g}')
          elif violation_amount > 0:
            print(f'  - Potential precision error at {boundary} boundary. Violation: {violation_amount:.4g}')

    return {'sum_of_radii': -np.inf}

  # --- Geometric check for circle overlaps ---
  if n > 1:
    has_overlap = False
    # Iterate over every unique pair of circles
    for i, j in itertools.combinations(range(n), 2):
      center_dist_sq = np.sum((centers[i] - centers[j]) ** 2)
      radii_sum_sq = (radii[i] + radii[j]) ** 2

      # Check if squared distance is less than squared sum of radii
      if center_dist_sq < radii_sum_sq:
        if not has_overlap: # Print header only once
          print('Error: Circles are overlapping.')
          has_overlap = True

        overlap_sq = radii_sum_sq - center_dist_sq
        # Distinguish between genuine overlap and touching circles (precision issue)
        if overlap_sq > TOLERANCE:
          print(f'  - Genuinely overlapping: Circles {i} and {j}. Squared overlap: {overlap_sq:.4g}')
        else:
          print(f'  - Potential precision error: Circles {i} and {j} are touching/minutely overlapping. Squared overlap: {overlap_sq:.4g}')

    if has_overlap:
      return {'sum_of_radii': -np.inf}

  if (
      radii.shape != (n,)
      or not np.isfinite(radii).all()
      or not (0 <= radii).all()
      or not np.isrealobj(radii)  # Added check for real numbers
  ):
    print('radii bad shape or contains non-real/non-finite values')
    return {'sum_of_radii': -np.inf}

  if _circles_overlap(centers, radii):
    print('circles overlap')
    return {'sum_of_radii': -np.inf}, {}

  print(
      f'Valid packing found with width={width}, height={height},'
      f' sum_radii={np.sum(radii)}'
  )

  print("The circles are disjoint and lie inside the rectangle.")
  return {'sum_of_radii': float(np.sum(radii))}


print(f"Construction has {len(centers_21)} circles.")
score = check_construction_rectangle(centers_21, radii_21, 21, width_21, height_21)
print(f"Construction sum of radii: {score['sum_of_radii']}")

In [ ]:


visualize_packing(centers_21, radii_21, width_21, height_21, "AlphaEvolve's 21-Circle Packing")

In [ ]:
#@title Code evolved by AlphaEvolve

def construct_packing(num_circles):
    """Searches for the best packing of circles in a rectangle of perimeter 4."""
    start_time = time.time()

    N = num_circles
    if N == 0:
        return np.zeros((0, 2)), np.zeros(0), 1.0, 1.0

    best_sum_r = -1.0
    best_packing = None

    # Indices for variables in the optimization vector x.
    idx_radii = slice(0, N)
    idx_centers_flat = slice(N, 3 * N)
    idx_width = 3 * N
    num_vars = 3 * N + 1

    min_dim = 0.01

    # Objective function and its Jacobian (Gradient)
    def objective(x):
        return -np.sum(x[idx_radii])

    def objective_jac(x):
        grad = np.zeros(num_vars)
        grad[idx_radii] = -1.0
        return grad

    # Define constraints and their Jacobians once. This significantly speeds up optimization.
    cons = []

    # Containment constraints
    for i in range(N):
        # C1_i: x_center - radius >= 0 => x[N + 2*i] - x[i] >= 0
        def fun_c1(x, i=i):
            return x[N + 2 * i] - x[i]

        def jac_c1(x, i=i):
            jac = np.zeros(num_vars)
            jac[i] = -1.0 # dr_i
            jac[N + 2*i] = 1.0 # dcx_i
            return jac

        cons.append({'type': 'ineq', 'fun': fun_c1, 'jac': jac_c1})

        # C2_i: width - x_center - radius >= 0 => x[3N] - x[N + 2*i] - x[i] >= 0
        def fun_c2(x, i=i):
            return x[idx_width] - x[N + 2*i] - x[i]

        def jac_c2(x, i=i):
            jac = np.zeros(num_vars)
            jac[i] = -1.0 # dr_i
            jac[N + 2*i] = -1.0 # dcx_i
            jac[idx_width] = 1.0 # dW
            return jac

        cons.append({'type': 'ineq', 'fun': fun_c2, 'jac': jac_c2})

        # C3_i: y_center - radius >= 0 => x[N + 2*i + 1] - x[i] >= 0
        def fun_c3(x, i=i):
            return x[N + 2*i + 1] - x[i]

        def jac_c3(x, i=i):
            jac = np.zeros(num_vars)
            jac[i] = -1.0 # dr_i
            jac[N + 2*i + 1] = 1.0 # dcy_i
            return jac

        cons.append({'type': 'ineq', 'fun': fun_c3, 'jac': jac_c3})

        # C4_i: height - y_center - radius >= 0 => (2.0 - W) - y_c - r >= 0
        def fun_c4(x, i=i):
            return (2.0 - x[idx_width]) - x[N + 2*i + 1] - x[i]

        def jac_c4(x, i=i):
            jac = np.zeros(num_vars)
            jac[i] = -1.0 # dr_i
            jac[N + 2*i + 1] = -1.0 # dcy_i
            jac[idx_width] = -1.0 # dW
            return jac

        cons.append({'type': 'ineq', 'fun': fun_c4, 'jac': jac_c4})

    # Disjointness constraints
    for i in range(N):
        for j in range(i + 1, N):
            # C_ij: (cx_i - cx_j)^2 + (cy_i - cy_j)^2 - (r_i + r_j)^2 >= 0
            def fun_disjoint(x, i=i, j=j):
                cx_i, cy_i = x[N + 2*i], x[N + 2*i + 1]
                cx_j, cy_j = x[N + 2*j], x[N + 2*j + 1]
                r_i, r_j = x[i], x[j]
                return (cx_i - cx_j)**2 + (cy_i - cy_j)**2 - (r_i + r_j)**2

            def jac_disjoint(x, i=i, j=j):
                jac = np.zeros(num_vars)
                cx_i, cy_i = x[N + 2*i], x[N + 2*i + 1]
                cx_j, cy_j = x[N + 2*j], x[N + 2*j + 1]
                r_i, r_j = x[i], x[j]

                r_sum = r_i + r_j
                dx = cx_i - cx_j
                dy = cy_i - cy_j

                # Derivatives
                jac[i] = -2.0 * r_sum # dr_i
                jac[j] = -2.0 * r_sum # dr_j

                jac[N + 2*i] = 2.0 * dx # dcx_i
                jac[N + 2*i + 1] = 2.0 * dy # dcy_i

                jac[N + 2*j] = -2.0 * dx # dcx_j
                jac[N + 2*j + 1] = -2.0 * dy # dcy_j

                return jac

            cons.append({'type': 'ineq', 'fun': fun_disjoint, 'jac': jac_disjoint})

    # Bounds for radii, centers, and width.
    # Max radius is 0.5 (in a 1x1 square).
    bounds = [(0.0, 0.55)] * N  # Radii (slightly loose upper bound)
    for _ in range(N):
        bounds.extend([(0.0, 2.0), (0.0, 2.0)])  # Center_x, Center_y
    bounds.append((min_dim, 2.0 - min_dim))  # Width

    trial_num = 0
    time_limit = 985.0 # Safety buffer for setup and final processing

    while time.time() - start_time < time_limit:
        trial_num += 1

        # --- Initialization Strategy (Fully Random Restart) ---

        # 1. Initialize Width/Height: Randomly choose width (and thus height) for every trial.
        # Exploiting symmetry, we only need to search for width in [min_dim, 1.0].
        # Bias width towards 1.0 (square-like) using a Beta distribution to explore promising aspect ratios more frequently.
        # Exploiting symmetry, we only need to search for width in [min_dim, 1.0].
        # Bias width towards 1.0 (square-like) using a Beta distribution to explore promising aspect ratios more frequently.
        # With a 15% probability, explore "long" rectangles (width near min_dim, i.e. tall rectangles).
        # Otherwise, focus on "squarish" rectangles (width near 1.0).
        if np.random.rand() < 0.15:
            # Beta(1,3) concentrates values towards 0, making width small.
            width_scaled = np.random.beta(1, 3)
        else:
            # Beta(3,1) concentrates values towards 1.0, making width close to 1.0.
            width_scaled = np.random.beta(3, 1)

        width = min_dim + (1.0 - min_dim) * width_scaled
        height = 2.0 - width

        # 2. Initialize Radii
        # Estimate based on area coverage and randomized factor to diversify starting points.
        max_r_theoretically = min(width, height) / 2.0
        initial_r_estimate = np.sqrt((width * height) / N) / 2.0
        r_factor = 0.5 + 0.4 * np.random.rand() # Between 0.5 and 0.9
        initial_r = max(0.001, min(initial_r_estimate * r_factor, max_r_theoretically * 0.8))
        initial_radii = np.full(N, initial_r)

        # 3. Initialize Centers (Grid-based initialization with noise)

        # Calculate approximate grid dimensions.
        grid_cols = int(np.ceil(np.sqrt(N*width/height)))
        grid_rows = int(np.ceil(N/grid_cols))

        # Ensure centers are well within the bounds initially, respecting initial_r.
        # Centers must be at least initial_r from any edge.
        x_min_c, x_max_c = initial_r, width - initial_r
        y_min_c, y_max_c = initial_r, height - initial_r

        # The ranges (x_min_c, x_max_c) and (y_min_c, y_max_c) are guaranteed to be valid and non-empty
        # because initial_r is capped at 0.8 * min(width/2, height/2).

        # Calculate grid coordinates. For a single row/column, center it.
        x_coords = np.linspace(x_min_c, x_max_c, grid_cols) if grid_cols > 1 else [width / 2.0]
        y_coords = np.linspace(y_min_c, y_max_c, grid_rows) if grid_rows > 1 else [height / 2.0]

        grid_points = np.array(list(itertools.product(x_coords, y_coords)))

        if len(grid_points) >= N:
            # Randomly select N points from the grid
            indices = np.random.choice(len(grid_points), N, replace=False)
            initial_centers = grid_points[indices]
        else:
            # Fallback to random initialization if grid points are insufficient, within valid center range.
            initial_centers = np.random.uniform(low=[x_min_c, y_min_c], high=[x_max_c, y_max_c], size=(N, 2))

        # Add noise to the centers based on initial_r to help break symmetry and explore.
        noise_scale = initial_r * 0.5 * (0.5 + 0.5 * np.random.rand()) # Between 0.25*r and 0.5*r

        initial_centers += (np.random.rand(N, 2) - 0.5) * noise_scale

        # Ensure centers remain within the *valid* range after adding noise.
        initial_centers[:, 0] = np.clip(initial_centers[:, 0], x_min_c, x_max_c)
        initial_centers[:, 1] = np.clip(initial_centers[:, 1], y_min_c, y_max_c)

        x0 = np.concatenate([initial_radii, initial_centers.flatten(), [width]])

        # --- Optimization ---

        # With analytical jacobians, we can afford tighter tolerance and more iterations for better local optima.
        res = optimize.minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons,
                                jac=objective_jac,
                                options={'maxiter': 4000, 'ftol': 1e-15, 'disp': False}) # Further increased maxiter and tightened ftol for extreme precision.

        if res.success and -res.fun > best_sum_r:
            best_sum_r = -res.fun
            radii = res.x[idx_radii]
            centers = res.x[idx_centers_flat].reshape(N, 2)
            width = res.x[idx_width]
            height = 2.0 - width
            # Apply a very small safety margin to ensure strict disjointness in external checks.
            # With a tight ftol (1e-12), a margin of 1e-12 is robust.
            # Apply a very small safety margin to ensure strict disjointness in external checks.
            # With ftol=1e-15, a margin of 1e-14 is often robust enough.
            radii *= (1.0 - 1e-14)
            best_packing = (centers, radii, width, height)
            # Increase logging precision for the score.
            logging.warning(
                f"Trial {trial_num}: New best: {best_sum_r:.10f} (w={width:.4f}, h={height:.4f}) at t={time.time()-start_time:.1f}s. Iterations: {res.nit}")

    if best_packing is None:
        # This should ideally not happen if N>0.
        logging.warning("Optimizer failed to find any packing. Returning a default safe packing.")
        width = 1.0
        height = 1.0
        # Return a valid packing with tiny radii if N>0.
        centers = np.full((N, 2), 0.5)
        radii = np.zeros(N)
        if N > 0:
             # Place one tiny circle to ensure it's not entirely empty if N>0
             radii[0] = 1e-9
             centers[0] = [0.5, 0.5]
        return centers, radii, width, height

    return best_packing


**Prompt used** (Note that the prompt mistakenly said AlphaEvolve shouldn't use the previous best construction, which likely made AlphaEvolve's task harder. Fortunately this problem is easy enough, and AlphaEvolve was able to solve it from scratch)

Act as an expert software developer. Your task is to iteratively improve the provided codebase. Your primary goal is to write a search function that has 1000 seconds to solve a packing problem.

The problem is to place num_circles disjoint disks into a rectangle of perimeter 4. Crucially, your function must also find the optimal width and height of this rectangle to maximize the sum of the disks' radii.

Your program will be evaluated with a function signature like this:
centers, radii, width, height = construct_packing(num_circles)

So you must write a construct_packing(num_circles) function that returns four things:

width, height: Two positive, finite floating-point numbers that define the dimensions of the rectangle. They must satisfy the constraint 2 * (width + height) == 4.0.

radii: A 1D NumPy array of num_circles non-negative, finite numbers.

centers: A 2D NumPy array of shape (num_circles, 2), where each row contains the (x, y) coordinates for the center of a circle.

The following geometric constraints will be checked automatically:

Containment: All circles must be fully contained within the rectangle defined by the corners (0, 0) and (width, height).

Disjointness: The circles must be disjoint. This will be checked with a snippet as follows:

Python

for i in range(n):
  for j in range(i + 1, n):
    dist = np.sqrt(np.sum((centers[i] - centers[j]) ** 2))
    if radii[i] + radii[j] > dist:
      return False # Overlap detected
Your code has to complete its search in a maximum of 1000 seconds. Since the geometry of the problem has changed from a square to a variable rectangle, you should not rely on previous solutions and should start your search from a new, random configuration.

You got this! I believe in you!!!

